In [1]:
import os
import zipfile
from google.colab import drive

# --- 1. MOUNT DRIVE ---
drive.mount('/content/drive')

# --- 2. SETUP PATHS ---
# Verify this path matches your Drive exactly
zip_path = '/content/drive/MyDrive/my_clean_cropped_data_splitted_and_resized.zip'
extract_path = '/content/dataset'

# --- 3. EXTRACT (Force Overwrite to ensure fresh start) ---
print(f"Unzipping {zip_path} to {extract_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Extraction complete.\n")

# --- 4. RUTHLESS AUDIT ---
# We need to know EXACTLY what we are working with.
base_dir = os.path.join(extract_path, 'my_clean_cropped_data_splitted_and_resized')

if not os.path.exists(base_dir):
    print(f"CRITICAL ERROR: Directory {base_dir} not found. Check your zip file structure.")
else:
    print(f"{'SPLIT':<20} {'CLASS':<15} {'COUNT':<10}")
    print("-" * 45)

    total_images = 0
    # Walk through Test and Train folders
    for split in os.listdir(base_dir):
        split_path = os.path.join(base_dir, split)
        if os.path.isdir(split_path):
            for class_name in os.listdir(split_path):
                class_path = os.path.join(split_path, class_name)
                if os.path.isdir(class_path):
                    count = len(os.listdir(class_path))
                    print(f"{split:<20} {class_name:<15} {count:<10}")
                    total_images += count

    print("-" * 45)
    print(f"TOTAL IMAGES FOUND: {total_images}")

Mounted at /content/drive
Unzipping /content/drive/MyDrive/my_clean_cropped_data_splitted_and_resized.zip to /content/dataset...
Extraction complete.

SPLIT                CLASS           COUNT     
---------------------------------------------
val                  surprised       8         
val                  sad             20        
val                  angry           12        
val                  happy           32        
val                  fear            23        
train_balanced_aug   surprised       200       
train_balanced_aug   sad             200       
train_balanced_aug   angry           200       
train_balanced_aug   happy           200       
train_balanced_aug   fear            200       
test                 surprised       8         
test                 sad             20        
test                 angry           12        
test                 happy           32        
test                 fear            23        
-----------------------------------

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# --- 1. CONFIGURATION ---
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 40  # Set high, EarlyStopping will cut it short
LR = 1e-4    # Low learning rate for fine-tuning

# --- 2. DATA LOADERS (With MobileNet Preprocessing) ---
# Note: We do NOT augment validation/test data, only rescale.
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    # Add slight on-the-fly augmentation just to be safe
    rotation_range=20,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

# Load Data
train_ds = train_datagen.flow_from_directory(
    '/content/dataset/my_clean_cropped_data_splitted_and_resized/train_balanced_aug',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_ds = val_test_datagen.flow_from_directory(
    '/content/dataset/my_clean_cropped_data_splitted_and_resized/val',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Don't shuffle val/test for consistency
)

test_ds = val_test_datagen.flow_from_directory(
    '/content/dataset/my_clean_cropped_data_splitted_and_resized/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# --- 3. BUILD BASELINE MOBILENETV2 ---
def build_baseline_mobilenet():
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    # Freeze base model initially?
    # For small datasets, better to unfreeze top layers or train all with low LR.
    # We will Train ALL with very low LR for best performance.
    base_model.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x) # Simple regularization
    outputs = Dense(5, activation='softmax')(x) # 5 Classes

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

model = build_baseline_mobilenet()

# --- 4. COMPILE & TRAIN ---
model.compile(
    optimizer=Adam(learning_rate=LR),
    loss='categorical_crossentropy', # Standard Loss (No Weights)
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

print("Starting Baseline Training...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

# --- 5. EVALUATE ---
print("\n--- FINAL TEST EVALUATION ---")
loss, acc = model.evaluate(test_ds)
print(f"Baseline MobileNetV2 Accuracy: {acc*100:.2f}%")

Found 1000 images belonging to 5 classes.
Found 95 images belonging to 5 classes.
Found 95 images belonging to 5 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Starting Baseline Training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
32/32 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.3857 - loss: 1.5698 - val_accuracy: 0.3789 - val_loss: 1.6863 - learning_rate: 1.0000e-04
Epoch 2/40
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 397ms/step - accuracy: 0.8198 - loss: 0.5238 - val_accuracy: 0.4842 - val_loss: 1.3091 - learning_rate: 1.0000e-04
Epoch 3/40
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 390ms/step - accuracy: 0.9319 - loss: 0.2515 - val_accuracy: 0.5368 - val_loss: 1.0554 - learning_rate: 1.0000e-04
Epoch 4/40
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 393ms/step - accuracy: 0.9597 - loss: 0.1511 - val_accuracy: 0.5368 - val_loss: 1.0022 - learning_rate: 1.0000e-04
Epoch 5/40
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 390ms/step - accuracy: 0.9873 - loss: 0.0764 - val_accuracy: 0.5579 - val_loss: 1.0182 - learning_rate: 1.0000e-04
Epoch 6/40
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 394ms/step - accuracy: 0.9899 - loss: 0.0566 - val_accuracy: 0.6632 - val_loss: 0.8866 - learning_rate: 1.0000e-04
Epoch 7/40
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 389ms/step - accur

In [3]:
# Save to the mounted Drive path
save_path = '/content/drive/MyDrive/mobilenet_baseline_best.h5'
model.save(save_path)
print(f"Model saved to {save_path}")

Model saved to /content/drive/MyDrive/mobilenet_baseline_best.h5


MobileV2 with coordinate attention

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import mixed_precision
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# --- 1. ENABLE MIXED PRECISION (Speed Hack) ---
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# --- 2. FAST DATA PIPELINE (Partner's Idea + Fixes) ---
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Define Paths
base_dir = '/content/dataset/my_clean_cropped_data_splitted_and_resized'
train_dir = os.path.join(base_dir, 'train_balanced_aug')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Load Datasets (Using 'categorical' to match our loss function)
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', shuffle=True, seed=42
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', shuffle=False
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', shuffle=False
)

# Augmentation Block (Runs on GPU)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Preprocessing Function (CORRECT MobileNet Scaling: -1 to 1)
def preprocess(images, labels):
    # tf.keras.applications.mobilenet_v2.preprocess_input expects [0, 255]
    # It converts to [-1, 1] automatically.
    return tf.keras.applications.mobilenet_v2.preprocess_input(images), labels

def preprocess_train(images, labels):
    images = data_augmentation(images)
    return preprocess(images, labels)

# Optimization (Cache & Prefetch)
train_ds = train_ds.map(preprocess_train, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)
test_ds = test_ds.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

# --- 3. RE-DEFINE COORDINATE ATTENTION ---
class CoordinateAttention(layers.Layer):
    def __init__(self, reduction_ratio=32, **kwargs):
        super(CoordinateAttention, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        channels = input_shape[-1]
        reduction_channels = max(8, channels // self.reduction_ratio)
        self.conv_reduction = layers.Conv2D(reduction_channels, kernel_size=1, use_bias=False)
        self.bn = layers.BatchNormalization()
        self.activation = layers.Activation('relu')
        self.conv_w = layers.Conv2D(channels, kernel_size=1, use_bias=False)
        self.conv_h = layers.Conv2D(channels, kernel_size=1, use_bias=False)
        super(CoordinateAttention, self).build(input_shape)

    def call(self, inputs):
        shape = tf.shape(inputs)
        h, w, c = shape[1], shape[2], shape[3]
        x_h = tf.reduce_mean(inputs, axis=2, keepdims=True)
        x_w = tf.reduce_mean(inputs, axis=1, keepdims=True)
        x_w_perm = tf.transpose(x_w, perm=[0, 2, 1, 3])
        x_cat = tf.concat([x_h, x_w_perm], axis=1)
        x_cat = self.conv_reduction(x_cat)
        x_cat = self.bn(x_cat)
        x_cat = self.activation(x_cat)
        x_h_prime, x_w_prime = tf.split(x_cat, num_or_size_splits=[h, w], axis=1)
        x_w_prime = tf.transpose(x_w_prime, perm=[0, 2, 1, 3])
        att_h = tf.nn.sigmoid(self.conv_w(x_h_prime))
        att_w = tf.nn.sigmoid(self.conv_h(x_w_prime))
        return inputs * att_h * att_w

# --- 4. BUILD MODEL (Unfrozen + CoordAtt) ---
def build_final_model():
    base_model = tf.keras.applications.MobileNetV2(
        weights='imagenet', include_top=False, input_shape=(224, 224, 3)
    )
    base_model.trainable = True # CRITICAL: Must be True to learn emotions

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = CoordinateAttention(reduction_ratio=32)(x) # The Novelty
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(5, activation='softmax', dtype='float32')(x)

    return models.Model(inputs, outputs, name="PRDA_MobileNetV2_Final")

model = build_final_model()

# --- 5. COMPILE & TRAIN ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1),
    ModelCheckpoint('/content/drive/MyDrive/prda_mobilenetv2_final.h5',
                    monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
]

print("Starting FINAL Training (Fast Pipe + CoordAtt + Unfrozen)...")
history = model.fit(train_ds, validation_data=val_ds, epochs=60, callbacks=callbacks)

# --- 6. METRICS ---
print("\n--- GENERATING FINAL REPORT ---")
Y_pred = model.predict(test_ds)
y_pred = np.argmax(Y_pred, axis=1)
y_true = np.concatenate([y for x, y in test_ds], axis=0) # Extract labels from dataset
y_true = np.argmax(y_true, axis=1)

print(classification_report(y_true, y_pred, target_names=['angry', 'fear', 'happy', 'sad', 'surprised']))

final_acc = np.sum(y_pred == y_true) / len(y_true)
print(f"FINAL TEST ACCURACY: {final_acc*100:.2f}%")

Found 1000 files belonging to 5 classes.
Found 95 files belonging to 5 classes.
Found 95 files belonging to 5 classes.
Starting FINAL Training (Fast Pipe + CoordAtt + Unfrozen)...
Epoch 1/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.3658 - loss: 1.5084   
Epoch 1: val_accuracy improved from -inf to 0.61053, saving model to /content/drive/MyDrive/prda_mobilenetv2_final.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 306s 5s/step - accuracy: 0.3696 - loss: 1.5046 - val_accuracy: 0.6105 - val_loss: 1.4011 - learning_rate: 1.0000e-04
Epoch 2/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9011 - loss: 0.7711
Epoch 2: val_accuracy did not improve from 0.61053
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.9007 - loss: 0.7661 - val_accuracy: 0.6000 - val_loss: 1.1677 - learning_rate: 1.0000e-04
Epoch 3/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9686 - loss: 0.3200
Epoch 3: val_accuracy improved from 0.61053 to 0.63158, saving model to /content/drive/MyDrive/prda_mobilenetv2_final.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.9687 - loss: 0.3176 - val_accuracy: 0.6316 - val_loss: 0.9912 - learning_rate: 1.0000e-04
Epoch 4/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9950 - loss: 0.1158
Epoch 4: val_accuracy improved from 0.63158 to 0.67368, saving model to /content/drive/MyDrive/prda_mobilenetv2_final.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.9949 - loss: 0.1149 - val_accuracy: 0.6737 - val_loss: 0.8832 - learning_rate: 1.0000e-04
Epoch 5/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9997 - loss: 0.0458
Epoch 5: val_accuracy improved from 0.67368 to 0.70526, saving model to /content/drive/MyDrive/prda_mobilenetv2_final.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.9997 - loss: 0.0456 - val_accuracy: 0.7053 - val_loss: 0.8336 - learning_rate: 1.0000e-04
Epoch 6/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 0.0246
Epoch 6: val_accuracy did not improve from 0.70526
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 1.0000 - loss: 0.0245 - val_accuracy: 0.7053 - val_loss: 0.8042 - learning_rate: 1.0000e-04
Epoch 7/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.0165
Epoch 7: val_accuracy improved from 0.70526 to 0.72632, saving model to /content/drive/MyDrive/prda_mobilenetv2_final.h5


...

32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.8842 - val_loss: 0.3129 - learning_rate: 1.0000e-04
Epoch 26/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 26: val_accuracy did not improve from 0.88421
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.8842 - val_loss: 0.2980 - learning_rate: 1.0000e-04
Epoch 27/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 9.8636e-04
Epoch 27: val_accuracy did not improve from 0.88421
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 1.0000 - loss: 9.8653e-04 - val_accuracy: 0.8842 - val_loss: 0.2895 - learning_rate: 1.0000e-04
Epoch 28/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 9.7074e-04
Epoch 28: val_accuracy did not improve from 0.88421
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 1.0000 - loss: 9.7077e-04 - val_accuracy: 0.8842 - val_loss: 0.2794 - learning_rate: 1.000

32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 1.0000 - loss: 9.1616e-04 - val_accuracy: 0.8947 - val_loss: 0.2694 - learning_rate: 1.0000e-04
Epoch 30/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 8.6238e-04
Epoch 30: val_accuracy did not improve from 0.89474
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 1.0000 - loss: 8.6093e-04 - val_accuracy: 0.8842 - val_loss: 0.2605 - learning_rate: 1.0000e-04
Epoch 31/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 8.2805e-04
Epoch 31: val_accuracy did not improve from 0.89474
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 1.0000 - loss: 8.2721e-04 - val_accuracy: 0.8737 - val_loss: 0.2530 - learning_rate: 1.0000e-04
Epoch 32/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 7.6101e-04
Epoch 32: val_accuracy did not improve from 0.89474
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 1.0000 - loss: 7.6145e-04 - val_accuracy: 0.8842 - val_loss: 0.2497 - learning

32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 1.0000 - loss: 4.3935e-04 - val_accuracy: 0.9053 - val_loss: 0.2181 - learning_rate: 1.0000e-04
Epoch 43/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 3.9655e-04
Epoch 43: val_accuracy did not improve from 0.90526
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 1.0000 - loss: 3.9722e-04 - val_accuracy: 0.9053 - val_loss: 0.2164 - learning_rate: 1.0000e-04
Epoch 44/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 3.9413e-04
Epoch 44: val_accuracy did not improve from 0.90526
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 1.0000 - loss: 3.9449e-04 - val_accuracy: 0.9053 - val_loss: 0.2153 - learning_rate: 1.0000e-04
Epoch 45/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 3.9544e-04
Epoch 45: val_accuracy improved from 0.90526 to 0.91579, saving model to /content/drive/MyDrive/prda_mobilenetv2_final.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 1.0000 - loss: 3.9541e-04 - val_accuracy: 0.9158 - val_loss: 0.2130 - learning_rate: 1.0000e-04
Epoch 46/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 3.7918e-04
Epoch 46: val_accuracy did not improve from 0.91579
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 1.0000 - loss: 3.7861e-04 - val_accuracy: 0.9158 - val_loss: 0.2126 - learning_rate: 1.0000e-04
Epoch 47/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 3.6252e-04
Epoch 47: val_accuracy did not improve from 0.91579
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 1.0000 - loss: 3.6129e-04 - val_accuracy: 0.9158 - val_loss: 0.2124 - learning_rate: 1.0000e-04
Epoch 48/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 3.4110e-04
Epoch 48: val_accuracy improved from 0.91579 to 0.92632, saving model to /content/drive/MyDrive/prda_mobilenetv2_final.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 1.0000 - loss: 3.4106e-04 - val_accuracy: 0.9263 - val_loss: 0.2114 - learning_rate: 1.0000e-04
Epoch 49/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 3.0190e-04
Epoch 49: val_accuracy did not improve from 0.92632
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 1.0000 - loss: 3.0341e-04 - val_accuracy: 0.9158 - val_loss: 0.2106 - learning_rate: 1.0000e-04
Epoch 50/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 3.1155e-04
Epoch 50: val_accuracy did not improve from 0.92632
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 1.0000 - loss: 3.1174e-04 - val_accuracy: 0.9158 - val_loss: 0.2084 - learning_rate: 1.0000e-04
Epoch 51/60
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 3.1280e-04
Epoch 51: val_accuracy did not improve from 0.92632
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 1.0000 - loss: 3.1259e-04 - val_accuracy: 0.9158 - val_loss: 0.2071 - learning

MobilenetV3

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import os
import zipfile
from google.colab import drive

# --- 1. SETUP & RECOVERY ---
# Re-mount Drive
drive.mount('/content/drive')

# Paths
zip_path = '/content/drive/MyDrive/my_clean_cropped_data_splitted_and_resized.zip'
extract_path = '/content/dataset'
base_dir = '/content/dataset/my_clean_cropped_data_splitted_and_resized'

# Check if data exists, if not, unzip it
if not os.path.exists(base_dir):
    print(f"Data not found. Unzipping {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Unzipping complete.")
else:
    print("Data already exists. Skipping unzip.")

train_dir = os.path.join(base_dir, 'train_balanced_aug')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# --- 2. GPU SPEED HACK ---
tf.keras.backend.clear_session() # clear old memory
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# --- 3. LOW RAM DATA PIPELINE ---
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Load Data
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', shuffle=True
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', shuffle=False
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', shuffle=False
)

# Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])

def augment_only(images, labels):
    return data_augmentation(images), labels

# Optimize: Prefetch ONLY (No .cache() to save RAM)
train_ds = train_ds.map(augment_only, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

# --- 4. BUILD MODEL (V3 SAFE MODE) ---
# Let Keras handle preprocessing internally
def build_mobilenet_v3_safe():
    base_model = tf.keras.applications.MobileNetV3Large(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3),
        include_preprocessing=True
    )
    base_model.trainable = True

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(5, activation='softmax', dtype='float32')(x)

    return models.Model(inputs, outputs, name="Benchmark_MobileNetV3_Safe")

model = build_mobilenet_v3_safe()

# --- 5. COMPILE & TRAIN ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Low LR for stability
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('/content/drive/MyDrive/mobilenetv3_safe_best.h5',
                    monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
]

print("Starting V3 Training...")
history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=callbacks)

# --- 6. REPORT ---
loss, acc = model.evaluate(test_ds)
print(f"\nFINAL V3 ACCURACY: {acc*100:.2f}%")

Mounted at /content/drive
Data not found. Unzipping /content/drive/MyDrive/my_clean_cropped_data_splitted_and_resized.zip...
Unzipping complete.
Found 1000 files belonging to 5 classes.
Found 95 files belonging to 5 classes.
Found 95 files belonging to 5 classes.
12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Starting V3 Training...
Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.2097 - loss: 2.2520   
Epoch 1: val_accuracy improved from -inf to 0.08421, saving model to /content/drive/MyDrive/mobilenetv3_safe_best.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 365s 6s/step - accuracy: 0.2094 - loss: 2.2518 - val_accuracy: 0.0842 - val_loss: 2.3427
Epoch 2/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.2228 - loss: 2.0700
Epoch 2: val_accuracy did not improve from 0.08421
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 195ms/step - accuracy: 0.2234 - loss: 2.0666 - val_accuracy: 0.0842 - val_loss: 2.2504
Epoch 3/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.2906 - loss: 1.8394
Epoch 3: val_accuracy improved from 0.08421 to 0.11579, saving model to /content/drive/MyDrive/mobilenetv3_safe_best.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 277ms/step - accuracy: 0.2901 - loss: 1.8397 - val_accuracy: 0.1158 - val_loss: 2.1672
Epoch 4/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.3033 - loss: 1.7954
Epoch 4: val_accuracy improved from 0.11579 to 0.13684, saving model to /content/drive/MyDrive/mobilenetv3_safe_best.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 268ms/step - accuracy: 0.3030 - loss: 1.7943 - val_accuracy: 0.1368 - val_loss: 2.0921
Epoch 5/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.3472 - loss: 1.6412
Epoch 5: val_accuracy did not improve from 0.13684
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 198ms/step - accuracy: 0.3474 - loss: 1.6409 - val_accuracy: 0.1368 - val_loss: 2.0233
Epoch 6/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.3684 - loss: 1.5420
Epoch 6: val_accuracy improved from 0.13684 to 0.14737, saving model to /content/drive/MyDrive/mobilenetv3_safe_best.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 303ms/step - accuracy: 0.3693 - loss: 1.5409 - val_accuracy: 0.1474 - val_loss: 1.9561
Epoch 7/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.3817 - loss: 1.5106
Epoch 7: val_accuracy improved from 0.14737 to 0.15789, saving model to /content/drive/MyDrive/mobilenetv3_safe_best.h5


...

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 292ms/step - accuracy: 0.8796 - loss: 0.4115 - val_accuracy: 0.9158 - val_loss: 0.4520
Epoch 36/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.8981 - loss: 0.3610
Epoch 36: val_accuracy improved from 0.91579 to 0.92632, saving model to /content/drive/MyDrive/mobilenetv3_safe_best.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 217ms/step - accuracy: 0.8980 - loss: 0.3601 - val_accuracy: 0.9263 - val_loss: 0.4316
Epoch 37/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.9209 - loss: 0.3143
Epoch 37: val_accuracy did not improve from 0.92632
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 265ms/step - accuracy: 0.9202 - loss: 0.3149 - val_accuracy: 0.9263 - val_loss: 0.4113
Epoch 38/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.9223 - loss: 0.3101
Epoch 38: val_accuracy improved from 0.92632 to 0.93684, saving model to /content/drive/MyDrive/mobilenetv3_safe_best.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 256ms/step - accuracy: 0.9221 - loss: 0.3102 - val_accuracy: 0.9368 - val_loss: 0.3932
Epoch 39/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.8959 - loss: 0.3044
Epoch 39: val_accuracy improved from 0.93684 to 0.95789, saving model to /content/drive/MyDrive/mobilenetv3_safe_best.h5


32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 215ms/step - accuracy: 0.8963 - loss: 0.3046 - val_accuracy: 0.9579 - val_loss: 0.3746
Epoch 40/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 0.8999 - loss: 0.3190
Epoch 40: val_accuracy did not improve from 0.95789
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 231ms/step - accuracy: 0.8999 - loss: 0.3186 - val_accuracy: 0.9579 - val_loss: 0.3575
Epoch 41/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.9255 - loss: 0.2698
Epoch 41: val_accuracy did not improve from 0.95789
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 228ms/step - accuracy: 0.9257 - loss: 0.2694 - val_accuracy: 0.9579 - val_loss: 0.3425
Epoch 42/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.9244 - loss: 0.2846
Epoch 42: val_accuracy did not improve from 0.95789
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 193ms/step - accuracy: 0.9251 - loss: 0.2830 - val_accuracy: 0.9579 - val_loss: 0.3321
Epoch 43/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - accuracy: 0.9353 - loss: 0.2563
Epoch 43: val_accuracy did 

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 307ms/step - accuracy: 0.9379 - loss: 0.2314 - val_accuracy: 0.9684 - val_loss: 0.2954
Epoch 46/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.9405 - loss: 0.2349
Epoch 46: val_accuracy did not improve from 0.96842
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 191ms/step - accuracy: 0.9402 - loss: 0.2347 - val_accuracy: 0.9579 - val_loss: 0.2852
Epoch 47/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.9468 - loss: 0.2137
Epoch 47: val_accuracy did not improve from 0.96842
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 195ms/step - accuracy: 0.9464 - loss: 0.2137 - val_accuracy: 0.9579 - val_loss: 0.2770
Epoch 48/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.9388 - loss: 0.2135
Epoch 48: val_accuracy did not improve from 0.96842
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 240ms/step - accuracy: 0.9391 - loss: 0.2134 - val_accuracy: 0.9579 - val_loss: 0.2689
Epoch 49/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.9383 - loss: 0.2081
Epoch 49: val_accuracy did

In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, mixed_precision
import pandas as pd
import os
import zipfile
from google.colab import drive

# --- 1. SETUP (MUST MATCH TRAINING ENVIRONMENT) ---
tf.keras.backend.clear_session()
# FIX: Re-enable Mixed Precision so the float16 weights load correctly
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

drive.mount('/content/drive')
test_dir = '/content/dataset/my_clean_cropped_data_splitted_and_resized/test'

# --- 2. DEFINE CUSTOM LAYER ---
@tf.keras.utils.register_keras_serializable()
class CoordinateAttention(layers.Layer):
    def __init__(self, reduction_ratio=32, **kwargs):
        super(CoordinateAttention, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        channels = input_shape[-1]
        reduction_channels = max(8, channels // self.reduction_ratio)
        self.conv_reduction = layers.Conv2D(reduction_channels, kernel_size=1, use_bias=False)
        self.bn = layers.BatchNormalization()
        self.activation = layers.Activation('relu')
        self.conv_w = layers.Conv2D(channels, kernel_size=1, use_bias=False)
        self.conv_h = layers.Conv2D(channels, kernel_size=1, use_bias=False)
        super(CoordinateAttention, self).build(input_shape)

    def call(self, inputs):
        shape = tf.shape(inputs)
        h, w = shape[1], shape[2]
        x_h = tf.reduce_mean(inputs, axis=2, keepdims=True)
        x_w = tf.reduce_mean(inputs, axis=1, keepdims=True)
        x_w_perm = tf.transpose(x_w, perm=[0, 2, 1, 3])
        x_cat = tf.concat([x_h, x_w_perm], axis=1)
        x_cat = self.activation(self.bn(self.conv_reduction(x_cat)))
        x_h_prime, x_w_prime = tf.split(x_cat, num_or_size_splits=[h, w], axis=1)
        x_w_prime = tf.transpose(x_w_prime, perm=[0, 2, 1, 3])
        att_h = tf.nn.sigmoid(self.conv_w(x_h_prime))
        att_w = tf.nn.sigmoid(self.conv_h(x_w_prime))
        return inputs * att_h * att_w

    def get_config(self):
        config = super().get_config()
        config.update({"reduction_ratio": self.reduction_ratio})
        return config

# --- 3. EVALUATION LOOP ---
models_paths = {
    "MobileNetV2 (Baseline)": "/content/drive/MyDrive/mobilenet_baseline_best.h5",
    "MobileNetV3 (SOTA)":     "/content/drive/MyDrive/mobilenetv3_safe_best.h5",
    "MobileNetV2 + CA (Ours)": "/content/drive/MyDrive/prda_mobilenetv2_final.h5"
}

# Raw Test Data
raw_test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir, image_size=(224, 224), batch_size=32, label_mode='categorical', shuffle=False
)

results = []
print("\n--- CALCULATING REAL LOSS ---")

for name, path in models_paths.items():
    if not os.path.exists(path):
        continue

    print(f"Evaluating {name}...")
    try:
        # Load model with Custom Object Scope
        with tf.keras.utils.custom_object_scope({'CoordinateAttention': CoordinateAttention}):
            model = load_model(path, compile=False) # Skip compile to avoid optimizer mismatch

        model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

        # Preprocessing Wrapper
        def preprocess(x, y):
            # V3 has internal preprocessing, others need manual
            if "MobileNetV3" in name:
                return x, y
            else:
                return tf.keras.applications.mobilenet_v2.preprocess_input(x), y

        loss, acc = model.evaluate(raw_test_ds.map(preprocess), verbose=0)
        results.append({
            "Model": name,
            "Test Loss": f"{loss:.4f}",
            "Accuracy": f"{acc*100:.2f}%"
        })

    except Exception as e:
        print(f"Error on {name}: {e}")

# --- 4. PRINT TABLE ---
print("\n" + "="*40)
print(pd.DataFrame(results).to_markdown(index=False))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 95 files belonging to 5 classes.

--- CALCULATING REAL LOSS ---
Evaluating MobileNetV2 (Baseline)...
Evaluating MobileNetV3 (SOTA)...
Error on MobileNetV3 (SOTA): Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: 3.0 (of type <class 'float'>)
Evaluating MobileNetV2 + CA (Ours)...

| Model                   |   Test Loss | Accuracy   |
|:------------------------|------------:|:-----------|
| MobileNetV2 (Baseline)  |      0.291  | 93.68%     |
| MobileNetV2 + CA (Ours) |      0.2328 | 93.68%     |


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
import os
import zipfile
from google.colab import drive

# --- 1. SETUP ---
tf.keras.backend.clear_session()
# Re-enable mixed precision because the model was trained with it
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

drive.mount('/content/drive')
test_dir = '/content/dataset/my_clean_cropped_data_splitted_and_resized/test'

# --- 2. RE-BUILD V3 EXACTLY AS TRAINED ---
def build_mobilenet_v3_safe():
    # We must match the training structure 100%
    base_model = tf.keras.applications.MobileNetV3Large(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3),
        include_preprocessing=True # This layer handles the 0-255 scaling
    )
    base_model.trainable = True

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(5, activation='softmax', dtype='float32')(x)

    return models.Model(inputs, outputs, name="Benchmark_MobileNetV3_Safe")

# --- 3. LOAD WEIGHTS & EVALUATE ---
print("\n--- FORCING V3 EVALUATION ---")

# 1. Build blank model
model_v3 = build_mobilenet_v3_safe()

# 2. Compile (Required for evaluate)
model_v3.compile(loss='categorical_crossentropy', metrics=['accuracy'])

# 3. Load Weights (Not load_model)
# This bypasses the architecture reconstruction bug
try:
    print("Loading weights...")
    model_v3.load_weights("/content/drive/MyDrive/mobilenetv3_safe_best.h5")
    print("✅ Weights loaded successfully.")
except Exception as e:
    print(f"❌ Weight load failed: {e}")

# 4. Evaluate
# Note: V3 includes preprocessing, so we pass raw images
raw_test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir, image_size=(224, 224), batch_size=32, label_mode='categorical', shuffle=False
)

loss, acc = model_v3.evaluate(raw_test_ds, verbose=1)

print("\n" + "="*30)
print(f"FINAL V3 LOSS:     {loss:.4f}")
print(f"FINAL V3 ACCURACY: {acc*100:.2f}%")
print("="*30)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- FORCING V3 EVALUATION ---
12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Loading weights...
✅ Weights loaded successfully.
Found 95 files belonging to 5 classes.


3/3 ━━━━━━━━━━━━━━━━━━━━ 66s 16s/step - accuracy: 0.9227 - loss: 0.3561

FINAL V3 LOSS:     0.3659
FINAL V3 ACCURACY: 91.58%
